In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import io
import numpy as np
import os

In [2]:
URL = 'https://github.com/ltgoslo/nor_dia_change/tree/main/subset1/data'
page = requests.get(URL)

soup = BeautifulSoup( page.content , 'html.parser')
find_by_class = soup.find_all('a', class_="Link--primary")[3:]

In [3]:
judgements = []
uses = []
for i in find_by_class:
    judgements.append("https://raw.githubusercontent.com/"+i['href']+"/judgments.csv")
    uses.append("https://raw.githubusercontent.com/"+i['href']+"/uses.csv")

In [4]:
judgements_df = pd.DataFrame()
for i in judgements:
    judgements_df = pd.concat([judgements_df, pd.read_csv(io.StringIO(requests.get(i.replace("/tree","")).content.decode('utf-8')), delimiter='\t')])

In [5]:
judgements_df["comment"].value_counts()

comment    13468
Name: comment, dtype: int64

In [6]:
uses_df = pd.DataFrame()
for i in uses:
    uses_df = pd.concat([uses_df, pd.read_csv(io.StringIO(requests.get(i.replace("/tree","")).content.decode('utf-8')), delimiter='\t')])

In [7]:
judgements_grouped_df_with_comment = judgements_df.groupby(['identifier1', 'identifier2', 'annotator', 'comment', 'lemma'])['judgment'].apply(list).reset_index(name='judgments')
# judgements_df['median_judgment'] = judgements_df['judgments'].apply(lambda x: np.nanmedian(list(x)))

# Remove pairs with nan median
# judgements_df = judgements_df[~judgements_df['median_judgment'].isnull()]
display(judgements_grouped_df_with_comment)

,identifier1,identifier2,annotator,comment,lemma,judgments
0,13268,13269,titare,comment,skjerm,[0]
1,13268,13271,HelleBol,comment,skjerm,[2]
2,13268,13272,HelleBol,comment,skjerm,[2]
3,13268,13273,HelleBol,comment,skjerm,[2]
4,13268,13273,titare,comment,skjerm,[0]
...,...,...,...,...,...,...
13463,14730,14723,HelleBol,comment,husrom,[4]
13464,14730,14724,titare,comment,husrom,[0]
13465,14730,14726,alexandra_w,comment,husrom,[4]
13466,14730,14728,titare,comment,husrom,[4]


In [8]:
# Aggregate use pairs and extract median column
judgements_grouped_df = judgements_df.groupby(['identifier1', 'identifier2', 'annotator', 'lemma'])['judgment'].apply(list).reset_index(name='judgment')
# judgements_df['median_judgment'] = judgements_df['judgments'].apply(lambda x: np.nanmedian(list(x)))

# Remove pairs with nan median
# judgements_df = judgements_df[~judgements_df['median_judgment'].isnull()]
display(judgements_grouped_df)

,identifier1,identifier2,annotator,lemma,judgment
0,13268,13269,titare,skjerm,[0]
1,13268,13271,HelleBol,skjerm,[2]
2,13268,13272,HelleBol,skjerm,[2]
3,13268,13273,HelleBol,skjerm,[2]
4,13268,13273,titare,skjerm,[0]
...,...,...,...,...,...
13463,14730,14723,HelleBol,husrom,[4]
13464,14730,14724,titare,husrom,[0]
13465,14730,14726,alexandra_w,husrom,[4]
13466,14730,14728,titare,husrom,[4]


In [9]:
judgements_grouped_df["comment"] = " "
judgements_grouped_df = judgements_grouped_df[["identifier1", "identifier2", "annotator", "judgment", "comment", "lemma"]]

In [10]:
judgements_grouped_df

,identifier1,identifier2,annotator,judgment,comment,lemma
0,13268,13269,titare,[0],,skjerm
1,13268,13271,HelleBol,[2],,skjerm
2,13268,13272,HelleBol,[2],,skjerm
3,13268,13273,HelleBol,[2],,skjerm
4,13268,13273,titare,[0],,skjerm
...,...,...,...,...,...,...
13463,14730,14723,HelleBol,[4],,husrom
13464,14730,14724,titare,[0],,husrom
13465,14730,14726,alexandra_w,[4],,husrom
13466,14730,14728,titare,[4],,husrom


In [11]:
for i in list(judgements_df["lemma"].value_counts().index):
    df = judgements_df[judgements_df["lemma"]==i]
    numpy_df = df.to_numpy()
    header = list(df.columns)
    numpy_df = np.vstack([header, numpy_df])
    if not os.path.exists(i):
        os.mkdir(i)
    np.savetxt(i+"/judgements.csv", numpy_df,fmt='%s', delimiter='\t')

In [12]:
for i in list(uses_df["lemma"].value_counts().index):
    df = uses_df[uses_df["lemma"]==i]
    numpy_df = df.to_numpy()
    header = list(df.columns)
    numpy_df = np.vstack([header, numpy_df])
    if not os.path.exists(i):
        os.mkdir(i)
    np.savetxt(i+"/uses.csv", numpy_df,fmt='%s', delimiter='\t')